In [1]:
import pandas as pd
from pathlib import Path
from config import *
import psycopg2

In [2]:
# Make a connection to our cloud PostgresSql database
conn = psycopg2.connect(database=DATABASE, user=USER, password=PASSWORD, host=HOST, port= PORT)
cursor = conn.cursor()

In [3]:
# Read in results_weather dataframe. Bring in only the columns that we need. 
cursor.execute("Select * FROM results_weather")
data = cursor.fetchall()
colnames = [desc[0] for desc in cursor.description]
colnames1 = ['raceId','resultId','driverId','constructorId','grid','position','year','round','weather_warm',
            'weather_cold','weather_dry','weather_wet','weather_cloudy']
results_weather_df = pd.DataFrame(data, columns=colnames)
results = results_weather_df[colnames1]
print(results.shape)
results.head()

(25040, 13)


,raceId,resultId,driverId,constructorId,grid,position,year,round,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy
0,18,1,1,1,1,1,2008,1,1,0,0,0,0
1,18,2,2,2,5,2,2008,1,1,0,0,0,0
2,18,3,3,3,7,3,2008,1,1,0,0,0,0
3,18,4,4,4,11,4,2008,1,1,0,0,0,0
4,18,5,5,1,3,5,2008,1,1,0,0,0,0


In [4]:
# Read in races dataframe. Bring in only the columns that we need. 
cursor.execute("Select * FROM races")
data = cursor.fetchall()
colnames = [desc[0] for desc in cursor.description]
colnames1 = ['raceId', 'circuitId']
races_df = pd.DataFrame(data, columns=colnames)
races= races_df[colnames1]
print(races.shape)
races.head()

(1058, 2)


,raceId,circuitId
0,1,1
1,2,2
2,3,17
3,4,3
4,5,4


## Join Tables

In [5]:
# Merge results with races on raceId.
data = (results.merge(races, left_on='raceId', right_on='raceId').reindex(
            columns=['year', 'round', 'resultId', 'raceId', 'circuitId', 'driverId', 
                     'constructorId','grid', 'position', 'weather_warm', 'weather_cold', 
                     'weather_dry', 'weather_wet', 'weather_cloudy' ]))
print(data.shape)
data.head()

(25040, 14)


,year,round,resultId,raceId,circuitId,driverId,constructorId,grid,position,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy
0,2008,1,1,18,1,1,1,1,1,1,0,0,0,0
1,2008,1,2,18,1,2,2,5,2,1,0,0,0,0
2,2008,1,3,18,1,3,3,7,3,1,0,0,0,0
3,2008,1,4,18,1,4,4,11,4,1,0,0,0,0
4,2008,1,5,18,1,5,1,3,5,1,0,0,0,0


In [6]:
print(data.shape)
data.head()

(25040, 14)


,year,round,resultId,raceId,circuitId,driverId,constructorId,grid,position,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy
0,2008,1,1,18,1,1,1,1,1,1,0,0,0,0
1,2008,1,2,18,1,2,2,5,2,1,0,0,0,0
2,2008,1,3,18,1,3,3,7,3,1,0,0,0,0
3,2008,1,4,18,1,4,4,11,4,1,0,0,0,0
4,2008,1,5,18,1,5,1,3,5,1,0,0,0,0


In [7]:
# Replace all "\\N" in position column to 0 
replace1 = {"\\N":0}
data["position"] = data["position"].replace(replace1)

# Convert 'position' from object to numeric
data['position'] = pd.to_numeric(data['position'])

## Other Transformations

In [8]:
# Add a new column titled "win". This column will map all wins to 1 and else 0 from "position" column
data['Win'] = data["position"]
data.Win = data.position.map(lambda x: 1 if x == 1 else 0)
print(data.shape)
data.head()

(25040, 15)


,year,round,resultId,raceId,circuitId,driverId,constructorId,grid,position,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,Win
0,2008,1,1,18,1,1,1,1,1,1,0,0,0,0,1
1,2008,1,2,18,1,2,2,5,2,1,0,0,0,0,0
2,2008,1,3,18,1,3,3,7,3,1,0,0,0,0,0
3,2008,1,4,18,1,4,4,11,4,1,0,0,0,0,0
4,2008,1,5,18,1,5,1,3,5,1,0,0,0,0,0


In [9]:
# Add a second new column titled "podium". 
#This column will map all podiums to 1,2,3 and else 0 from "position" column

podium = [1,2,3]

def podium_order(x):
    if x in podium:
        return x
    else:
        return 0
data["podium"] = data["position"].apply(podium_order)    
print(data.shape)
data.head()

(25040, 16)


,year,round,resultId,raceId,circuitId,driverId,constructorId,grid,position,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,Win,podium
0,2008,1,1,18,1,1,1,1,1,1,0,0,0,0,1,1
1,2008,1,2,18,1,2,2,5,2,1,0,0,0,0,0,2
2,2008,1,3,18,1,3,3,7,3,1,0,0,0,0,0,3
3,2008,1,4,18,1,4,4,11,4,1,0,0,0,0,0,0
4,2008,1,5,18,1,5,1,3,5,1,0,0,0,0,0,0


In [10]:
# We will filter the data for years 2000 to 2019

mldata = data[(data['year']<=2019) & (data['year']>=2000)]
print(mldata.shape)
mldata.head()

(7940, 16)


,year,round,resultId,raceId,circuitId,driverId,constructorId,grid,position,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,Win,podium
0,2008,1,1,18,1,1,1,1,1,1,0,0,0,0,1,1
1,2008,1,2,18,1,2,2,5,2,1,0,0,0,0,0,2
2,2008,1,3,18,1,3,3,7,3,1,0,0,0,0,0,3
3,2008,1,4,18,1,4,4,11,4,1,0,0,0,0,0,0
4,2008,1,5,18,1,5,1,3,5,1,0,0,0,0,0,0


In [11]:
# Exporting mldata into json file
mldata.to_json(r'H:\UoFT_Bootcamp\Module_20\Capstone-Project\Dashboard-Code\data.json')


In [13]:
# We will upload our final mldata back into our cloud Postgress database. Create engine to upload data
connect = f"postgres://postgres:{PASSWORD}@{HOST}:{PORT}/postgres"
from sqlalchemy import create_engine
engine = create_engine(connect)

In [14]:
# Upload mldata table to database
mldata.to_sql(name='mldata', con=engine, if_exists='replace')